先看分配到了什么GPU

First we see what gpu we have

In [18]:
!nvidia-smi

Tue Nov  9 12:38:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    37W / 250W |   6315MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
!pip install transformers
!pip install datasets
!pip install sklearn

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


将微调后的模型上传到huggingface里面

In [21]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-crendential store but this isn't the helper defined on your machine.
You will have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal to set it as the default

git config --global credential.helper store


In [22]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [24]:
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from datasets import load_metric
from sklearn.model_selection import train_test_split

data preprocess 数据预处理

先把两个json里面的内容读取成pandas的格式，再拼接成一个大数据集

In [26]:
import pandas as pd
import os
data = pd.read_json('Sarcasm_Headlines_Dataset.json', lines=True)
data_v2 = pd.read_json('Sarcasm_Headlines_Dataset_v2.json', lines=True)
all_data = pd.concat([data,data_v2] , ignore_index = True , axis =0)
print(all_data.shape)

(55328, 3)


In [27]:
all_data.info

<bound method DataFrame.info of                                             article_link  ... is_sarcastic
0      https://www.huffingtonpost.com/entry/versace-b...  ...            0
1      https://www.huffingtonpost.com/entry/roseanne-...  ...            0
2      https://local.theonion.com/mom-starting-to-fea...  ...            1
3      https://politics.theonion.com/boehner-just-wan...  ...            1
4      https://www.huffingtonpost.com/entry/jk-rowlin...  ...            0
...                                                  ...  ...          ...
55323  https://www.theonion.com/jews-to-celebrate-ros...  ...            1
55324  https://local.theonion.com/internal-affairs-in...  ...            1
55325  https://www.huffingtonpost.com/entry/andrew-ah...  ...            0
55326  https://www.theonion.com/mars-probe-destroyed-...  ...            1
55327  https://www.theonion.com/dad-clarifies-this-no...  ...            1

[55328 rows x 3 columns]>

建一个空的DataFrame用来保存数据

In [32]:
df = pd.DataFrame()
df['text'] = all_data['headline']
df['label'] = all_data['is_sarcastic']

划分数据集

In [33]:
from sklearn.model_selection import train_test_split

df_sample = df.sample(frac = 1)

train_df,test_df = train_test_split(df_sample,test_size = 0.3)

print(train_df.shape)
print(test_df.shape)

(38729, 2)
(16599, 2)


In [34]:
train_df2 = pd.DataFrame({
    'text': train_df['text'].replace(r'\n', ' ', regex=True),
    'label': train_df['label']
})

test_df2 = pd.DataFrame({
    'text': test_df['text'].replace(r'\n', ' ', regex=True),
    'label': test_df['label']
})

转成列表的格式

In [35]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_df2['text'], train_df2['label'], test_size=.1)
test_texts = test_df2['text']
test_labels = test_df2['label']
train_texts = list(train_texts)
val_texts = list(val_texts)
test_texts = list(test_texts)
train_labels = list(train_labels)
val_labels = list(val_labels)
test_labels = list(test_labels)

In [36]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



制作数据集

In [38]:
import torch

class SarcasmDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SarcasmDataset(train_encodings, train_labels)
val_dataset = SarcasmDataset(val_encodings, val_labels)
test_dataset = SarcasmDataset(test_encodings, test_labels)

define metric

In [39]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy", "f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

fine-tune roberta 微调roberta模型



In [41]:
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
training_args = TrainingArguments(
    "roberta-scarcasm-discriminator",          # output directory
    evaluation_strategy = "epoch",
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    save_total_limit=3,
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()


loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https://huggingface.co/roberta-base/resolve/main/pytorch_model.bin from 

Epoch,Training Loss,Validation Loss,Accuracy
1,0.069600,0.355612,0.914020
2,0.114000,0.211543,0.941905
3,0.002300,0.164517,0.966951
4,0.001200,0.164033,0.970565


Saving model checkpoint to roberta-scarcasm-discriminator/checkpoint-500
Configuration saved in roberta-scarcasm-discriminator/checkpoint-500/config.json
Model weights saved in roberta-scarcasm-discriminator/checkpoint-500/pytorch_model.bin
Saving model checkpoint to roberta-scarcasm-discriminator/checkpoint-1000
Configuration saved in roberta-scarcasm-discriminator/checkpoint-1000/config.json
Model weights saved in roberta-scarcasm-discriminator/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to roberta-scarcasm-discriminator/checkpoint-1500
Configuration saved in roberta-scarcasm-discriminator/checkpoint-1500/config.json
Model weights saved in roberta-scarcasm-discriminator/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to roberta-scarcasm-discriminator/checkpoint-2000
Configuration saved in roberta-scarcasm-discriminator/checkpoint-2000/config.json
Model weights saved in roberta-scarcasm-discriminator/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [

TrainOutput(global_step=8716, training_loss=0.14337734355059756, metrics={'train_runtime': 1354.6061, 'train_samples_per_second': 102.926, 'train_steps_per_second': 6.434, 'total_flos': 4872093189864960.0, 'train_loss': 0.14337734355059756, 'epoch': 4.0})

In [42]:
trainer.push_to_hub("XSY/roberta-scarcasm-discriminator")

Saving model checkpoint to roberta-scarcasm-discriminator
Configuration saved in roberta-scarcasm-discriminator/config.json
Model weights saved in roberta-scarcasm-discriminator/pytorch_model.bin
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.38k/476M [00:00<?, ?B/s]

To https://huggingface.co/XSY/roberta-scarcasm-discriminator
   fc10289..54051f3  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9705654531371031}]}
To https://huggingface.co/XSY/roberta-scarcasm-discriminator
   54051f3..86c5e88  main -> main



'https://huggingface.co/XSY/roberta-scarcasm-discriminator/commit/54051f3d8b2163ef7a7101f88ae13a56310902c4'